In [ ]:
import os
import logging
from logger import setup_logger
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph

# Load environment variables
load_dotenv()

# Set up API keys and environment variables
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
WORKING_DIRECTORY = os.getenv('WORKING_DIRECTORY', './data_storage/')

# Validate critical environment variables
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
if not LANGCHAIN_API_KEY:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment variables.")

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-Agent Data Analysis System"

# Set up logger
logger = setup_logger()

# Initialize language models
try:
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=4096)
    process_llm = ChatOpenAI(model="gpt-4o", temperature=0, max_tokens=4096)
    json_llm = ChatOpenAI(
        model="gpt-4o-mini",
        model_kwargs={"response_format": {"type": "json_object"}},
        temperature=0,
        max_tokens=4096
    )
    logger.info("Language models initialized successfully.")
except Exception as e:
    logger.error(f"Error initializing language models: {str(e)}")
    raise

# Ensure working directory exists
if not os.path.exists(WORKING_DIRECTORY):
    os.makedirs(WORKING_DIRECTORY)
    logger.info(f"Created working directory: {WORKING_DIRECTORY}")

logger.info("Initialization complete.")

In [ ]:
from state import State
from node import agent_node,human_choice_node,note_agent_node
from create_agent import create_agent,create_supervisor
from router import QualityReview_router,hypothesis_router,process_router

In [ ]:
# Create state graph for the workflow
workflow = StateGraph(State)

In [ ]:
members = ["Hypothesis","Process","Visualization", "Search", "Coder", "Report", "QualityReview"]

In [ ]:
from tools.internet import google_search,FireCrawl_scrape_webpages
from tools.basetool import execute_code,execute_command
from tools.FileEdit import create_document,read_document,edit_document,collect_data
from langchain.agents import load_tools
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
hypothesis_agent = create_agent(
llm, 
[collect_data,wikipedia,google_search,FireCrawl_scrape_webpages]+load_tools(["arxiv"],),
'''
As an esteemed expert in data analysis, 
your task is to formulate a set of research hypotheses and outline the steps to be taken based on the information table provided. 
Utilize statistics, machine learning, deep learning, and artificial intelligence in developing these hypotheses. 
Your hypotheses should be precise, achievable, professional, and innovative. To ensure the feasibility and uniqueness of your hypotheses, 
thoroughly investigate relevant information. For each hypothesis, include ample references to support your claims.

Upon analyzing the information table, you are required to:

1.Formulate research hypotheses that leverage statistics, machine learning, deep learning, and AI techniques.
2.Outline the steps involved in testing these hypotheses.
3.Verify the feasibility and uniqueness of each hypothesis through a comprehensive literature review.
At the conclusion of your analysis, present the complete research hypotheses, elaborate on their uniqueness and feasibility, 
and provide relevant references to support your assertions. Please answer in structured way to enhance readability.
Just answer a research hypothesis.
''',
members,WORKING_DIRECTORY)

In [ ]:
process_agent = create_supervisor(
    process_llm,
    '''
    As research supervisor, coordinate a comprehensive data analysis process resulting in a complete report. Manage "Visualization", "Search", "Coder", and "Report" agents to achieve the following:

    1. Validate and refine the research hypothesis.
    2. Conduct thorough data analysis with well-documented code.
    3. Produce a complete research report including:
       - Introduction
       - Hypothesis
       - Methodology
       - Results with relevant visualizations
       - Discussion
       - Conclusion
       - References

    For each step:
    1. Define clear objectives and outcomes.
    2. Assign specific tasks to appropriate agent(s).
    3. Review and integrate agent outputs.
    4. Ensure quality and relevance of all contributions.

    Agent Guidelines:
    - Visualization: Create and explain insightful data visualizations.
    - Search: Gather relevant information and compile references.
    - Coder: Develop efficient, documented data analysis code.
    - Report: Draft and refine report sections.

    Workflow:
    1. Plan the analysis process.
    2. Assign and oversee agent tasks.
    3. Integrate insights across agents.
    4. Refine the analysis based on emerging results.
    5. Compile the final report.

    After each agent task, evaluate output and determine the next step. Respond with the next agent to act or "FINISH" only when:
    1. Hypothesis is thoroughly tested.
    2. Analysis is complete with documented code.
    3. All visualizations are created and explained.
    4. Comprehensive report is written with all sections.
    5. Reference list is complete.
    6. All components are integrated cohesively.

    Ensure a logical flow, address all hypothesis aspects, and deliver a cohesive, insightful final report meeting high academic standards.
    ''',
    ["Visualization", "Search", "Coder", "Report"],
)

In [ ]:


visualization_agent = create_agent(
    llm, 
    [read_document, execute_code, execute_command],
    '''
    You are a data visualization expert responsible for creating insightful visualizations. Your tasks include:
    1. Analyzing data and creating appropriate visualizations.
    2. Interpreting visual patterns and trends in the data.
    3. Suggesting improvements for existing visualizations.
    4. Explaining the significance of visual findings.

    When asked to create a visualization, provide Python code that uses libraries like matplotlib, seaborn, or plotly. Ensure your code is complete and ready to execute. Include comments to explain key parts of your code.
    ''',
    members,WORKING_DIRECTORY
    )

code_agent = create_agent(
    llm,
    [read_document,execute_code, execute_command],
    '''You are an expert Python programmer specializing in data analysis and visualization. Your responsibilities include:
    1. Writing efficient and clean Python code for data analysis tasks.
    2. Implementing statistical methods and machine learning algorithms.
    3. Creating informative and visually appealing data visualizations.
    4. Debugging and optimizing existing code.
    5. Ensuring code readability and adherence to PEP 8 standards.

    Important guidelines for visualization and image-related tasks:
    1. Always save generated plots or images to files.
    2. Use a consistent naming convention for saved files, e.g., 'plot_YYYYMMDD_HHMMSS.png'.
    3. Include the file path of the saved image in your code output.
    4. Use matplotlib's 'Agg' backend to ensure plots can be saved without a GUI.

    When asked to write code, provide only valid Python code as a string. Do not include any explanatory text outside of code comments. Ensure all string literals are properly terminated. The code should be ready to execute without any modifications.

    Here's a template for saving plots:

    ```python
    import matplotlib
    matplotlib.use('Agg')  # Use the 'Agg' backend
    import matplotlib.pyplot as plt
    from datetime import datetime

    # Your plotting code here
    # ...

    # Save the plot
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    filename = f'plot_name_timestamp.png'
    plt.savefig(filename)
    plt.close()  # Close the plot to free up memory

    print(f"Plot saved as: filename")
    ```

    Always include similar code for saving any generated visualizations or images.
    ''',
    members,WORKING_DIRECTORY
)

searcher_agent= create_agent(
    llm,
    [read_document, collect_data,wikipedia,google_search,FireCrawl_scrape_webpages]+load_tools(["arxiv"],),
    '''You are a skilled research assistant responsible for gathering relevant information. Your tasks include:
    1. Conducting comprehensive literature reviews using academic databases.
    2. Searching the web for current and relevant information on specified topics.
    3. Evaluating the credibility and relevance of sources.
    4. Summarizing key findings from your searches.

    When providing information, always cite your sources. Prioritize peer-reviewed academic sources, but also include reputable web sources when appropriate. Present your findings in a clear, concise manner.''',
    members,WORKING_DIRECTORY
    )

report_agent = create_agent(
    llm, 
    [create_document, read_document, edit_document], 
    '''You are an experienced scientific writer responsible for creating comprehensive research reports. Your primary goal is to produce a thorough and well-structured data analysis report. Your tasks include:

    1. Formulating and clearly stating the research hypothesis based on the given data and analysis objectives.
    2. Detailing the methodology used for data collection and analysis.
    3. Synthesizing information from various sources into a coherent report.
    4. Structuring the report logically with clear sections:
       - Executive Summary
       - Introduction (including the research hypothesis)
       - Methodology
       - Results
       - Discussion
       - Conclusion
       - References
    5. Ensuring clarity and precision in scientific writing.
    6. Incorporating and explaining relevant data visualizations (charts, graphs, tables) to support your findings.
    7. Conducting multi-faceted analysis, considering various aspects of the data.
    8. Citing sources accurately using APA citation style.

    When writing the report:
    - Begin with a clear statement of the research hypothesis in the introduction.
    - In the methodology section, provide a detailed explanation of data collection methods, analysis techniques, and any statistical tests used.
    - Present results objectively, using appropriate visualizations to illustrate key findings.
    - In the discussion section, interpret the results in the context of the research hypothesis and existing literature.
    - Include a variety of charts and graphs that effectively communicate different aspects of the data analysis.
    - Ensure all visualizations are properly labeled, captioned, and referenced in the text.
    - Maintain an objective tone throughout and support all claims with evidence from your analysis or cited sources.
    - Use clear, concise language suitable for an academic audience.
    - Conclude by summarizing key findings and their implications, relating back to the initial hypothesis.
    - Provide a comprehensive list of references for all sources cited in the report.

    Remember to integrate insights from other team members (Visualization, Search, and Coder) into your report. Use their outputs to strengthen your analysis and support your conclusions.''',
    members,WORKING_DIRECTORY
)

In [ ]:
quality_review_agent=create_agent(
    llm, 
    [create_document,read_document,edit_document], 
    '''
    You are a meticulous quality control expert responsible for reviewing and ensuring the high standard of all research outputs. Your tasks include:
    1. Critically evaluating the content, methodology, and conclusions of research reports.
    2. Checking for consistency, accuracy, and clarity in all documents.
    3. Identifying areas that need improvement or further elaboration.
    4. Ensuring adherence to scientific writing standards and ethical guidelines.

    After your review, if revisions are needed, respond with 'REVISION' as a prefix, set needs_revision=True, and provide specific feedback on parts that need improvement. If no revisions are necessary, respond with 'CONTINUE' as a prefix and set needs_revision=False.
    ''',
    members,WORKING_DIRECTORY
    )
                        

In [ ]:
from create_agent import create_note_agent
note_agent=create_note_agent(
    json_llm, 
    [read_document], 
    '''
    You are a detail-oriented research process note-taker. Your primary responsibility is to summarize and document the actions and findings of other team members. Your tasks include:
    1. Observing and recording key activities and decisions made by the research team.
    2. Summarizing complex information into clear, concise notes.
    3. Organizing notes in a structured, easily retrievable format.
    4. Highlighting important insights, breakthroughs, or challenges encountered.
    5. Only response JSON object
    ''',
    )

In [ ]:
workflow.add_node("Hypothesis", lambda state: agent_node(state, hypothesis_agent, "hypothesis_agent"))
workflow.add_node("Process", lambda state: agent_node(state, process_agent, "process_agent"))
workflow.add_node("Visualization", lambda state: agent_node(state, visualization_agent, "visualization_agent"))
workflow.add_node("Search", lambda state: agent_node(state, searcher_agent, "searcher_agent"))
workflow.add_node("Coder", lambda state: agent_node(state, code_agent, "code_agent"))
workflow.add_node("Report", lambda state: agent_node(state, report_agent, "report_agent"))
workflow.add_node("QualityReview", lambda state: agent_node(state, quality_review_agent, "quality_review_agent"))
workflow.add_node("NoteTaker", lambda state: note_agent_node(state, note_agent, "note_agent"))
workflow.add_node("HumanChoice", human_choice_node)

In [ ]:
from langgraph.graph import END, START

workflow.add_edge("Hypothesis", "HumanChoice")
workflow.add_conditional_edges(
    "HumanChoice",
    hypothesis_router,
    {
        "Hypothesis": "Hypothesis",
        "Process": "Process"
    }
)

workflow.add_conditional_edges(
    "Process",
    process_router,
    {
        "Coder": "Coder",
        "Search": "Search",
        "Visualization": "Visualization",
        "Report": "Report",
        "Process": "Process",
        "END": END
    }
)

for member in ["Visualization",'Search','Coder','Report']:
    workflow.add_edge(member, "QualityReview")

In [ ]:
workflow.add_conditional_edges(
    "QualityReview",
    QualityReview_router,
    {
        'Visualization': "Visualization",
        'Search': "Search",
        'Coder': "Coder",
        'Report': "Report",
        'NoteTaker': "NoteTaker",
    }
)
workflow.add_edge("NoteTaker", "Process")

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
workflow.add_edge(START, "Hypothesis")
memory = MemorySaver()
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
userInput = '''
datapath:./data_storage/OnlineSalesData.csv
1. Generate testable hypotheses based on labels or variable names given by the data.

2. Build a statistical model of these testable hypotheses.

3. Use data to estimate statistical models

4. Use the results of statistical model estimation to verify the hypothesis.

5. Generate reports or charts based on verified hypotheses.
'''

In [ ]:
from langchain_core.messages import HumanMessage
events = graph.stream(
    {
        "messages": [
            HumanMessage(
                content=userInput
            ),
        ],
        "hypothesis": "",
        "process_decision":"",
        "process": "",
        "visualization_state": "",
        "searcher_state": "",
        "code_state": "",
        "report_section": "",
        "quality_review": "",
        "needs_revision": False,
        "last_sender": "",
    },
    {"configurable": {"thread_id": "1"}, "recursion_limit": 300},
    stream_mode="values",
    debug=False
)

In [ ]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message,end='',flush=True)
        else:
            message.pretty_print()

In [ ]:
print_stream(events)